#Installers

In [ ]:
!pip install transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

#HUGGING FACE TOKEN

In [ ]:
from huggingface_hub import login
login()

#MOUNT DRIVE

In [ ]:
from google.colab import drive
import pandas as pd

# Monta Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


#Minerva 1B & 3B

## Minerva 1B pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "sapienzanlp/Minerva-1B-base-v1.0"

#Setup to minerva 1b version
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")




/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/795k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.01G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import torch
import json
tqdm.pandas()


df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_cleaned.csv")



# Costruzione del prompt stile conversazione
def build_prompt_conversation(sentence):
    return f"{sentence.strip()}\nTraduzione:"




# Funzione di generazione con il modello
def get_translation(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    full = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full[len(prompt):].strip()


# Costruzione prompt e generazione traduzioni
df["prompt"] = df["Sentence"].apply(build_prompt_conversation)
df["modern_translation"] = df["prompt"].progress_apply(get_translation)

# Esporta in JSONL
output_path = "/content/minerva_1b.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        json.dump({
            "author": row["Author"],
            "date": row["Date"],
            "region": row["Region"],
            "archaic_sentence": row["Sentence"],
            "modern_translation": row["modern_translation"]
        }, f, ensure_ascii=False)
        f.write("\n")

print(f" File JSONL salvato: {output_path}")


  0%|          | 0/30 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

 File JSONL salvato: /content/minerva_batch_30.jsonl


## Minerva 3B pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "sapienzanlp/Minerva-3B-base-v1.0"

#Setup to minerva 3b version
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import torch
import json
tqdm.pandas()


df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_cleaned.csv")



# Costruzione del prompt stile conversazione
def build_prompt_conversation(sentence):
    return f"{sentence.strip()}\nTraduzione:"




# Funzione di generazione con il modello
def get_translation(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    full = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full[len(prompt):].strip()


# Costruzione prompt e generazione traduzioni
df["prompt"] = df["Sentence"].apply(build_prompt_conversation)
df["modern_translation"] = df["prompt"].progress_apply(get_translation)

# Esporta in JSONL
output_path = "/content/minerva_3b.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        json.dump({
            "author": row["Author"],
            "date": row["Date"],
            "region": row["Region"],
            "archaic_sentence": row["Sentence"],
            "modern_translation": row["modern_translation"]
        }, f, ensure_ascii=False)
        f.write("\n")

print(f" File JSONL salvato: {output_path}")

I initially tested Minerva-1B and Minerva-3B, but the results were not satisfactory. Both models showed limited performance, struggling to produce fluent and semantically accurate translations. Additionally, they were noticeably slower in generation, especially when using few-shot prompts. Based on these limitations, I decided to proceed with Minerva-7B for more reliable and efficient outputs.

#Minerva7b Analysis

##STEP 1: Test zero hot, few shot and judge


Pipeline for Minerva 7b

In [ ]:
import torch
from transformers import pipeline

minerva = pipeline(
    task="text-generation",
    model="sapienzanlp/Minerva-7B-instruct-v1.0",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def minerva_zero_shot(sentence):
    prompt = f"Traduci questa frase dall'italiano antico all'italiano moderno:\n\n\"{sentence}\"\n\nTraduzione:"
    response = minerva(prompt, max_new_tokens=150, do_sample=False)[0]["generated_text"]
    return response

In [ ]:
def minerva_few_shot(sentence):
    prompt = (
        "Traduci le seguenti frasi dall'italiano antico all'italiano moderno.\n\n"
        "Antico: \"Ché 'l sol che v'era allor, or non è più.\"\n"
        "Moderno: \"Perché il sole che c'era allora, ora non c'è più.\"\n\n"
        "Antico: \"Poi che 'l mio duol non vuol mostrar altrui.\"\n"
        "Moderno: \"Dato che il mio dolore non vuole mostrarsi agli altri.\"\n\n"
        f"Antico: \"{sentence}\"\n"
        "Moderno:"
    )
    response = minerva(prompt, max_new_tokens=150, do_sample=False)[0]["generated_text"]
    return response


In [ ]:
archaic_sentence = "Ché 'l sol che v'era allor, or non è più."
print(" Zero-shot:", minerva_zero_shot(archaic_sentence))
print(" Few-shot:", minerva_few_shot(archaic_sentence))


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 Zero-shot: Traduci questa frase dall'italiano antico all'italiano moderno:

"Ché 'l sol che v'era allor, or non è più."

Traduzione: "Perché il sole che c'era allora non è più".

La traduzione mantiene il significato originale della frase in italiano antico. La parola "allora" viene sostituita con "allor", e la preposizione "che" viene sostituita con "il". Inoltre, vengono mantenuti i pronomi personali "ché" (perché) e "non è più" per mantenere l'enfasi sulla causa del cambiamento.
 Few-shot: Traduci le seguenti frasi dall'italiano antico all'italiano moderno.

Antico: "Ché 'l sol che v'era allor, or non è più."
Moderno: "Perché il sole che c'era allora, ora non c'è più."

Antico: "Poi che 'l mio duol non vuol mostrar altrui."
Moderno: "Dato che il mio dolore non vuole mostrarsi agli altri."

Antico: "Ché 'l sol che v'era allor, or non è più."
Moderno: "Perché il sole che c'era allora, ora non c'è più."

Antico: "E se 'l mondo sapesse il cor ch'io sento!"
Moderno: "Se il mondo sapesse

In [ ]:
test_sentence = "Et dall’altra parte Aiaces era uno cavaliere franco e prode all’arme."
print("Zero-shot:", minerva_zero_shot(test_sentence))
print("Few-shot:", minerva_few_shot(test_sentence))


Zero-shot: Traduci questa frase dall'italiano antico all'italiano moderno:

"Et dall’altra parte Aiaces era uno cavaliere franco e prode all’arme."

Traduzione: E dall'altro lato Aiace era un cavaliere franco e valoroso.

Spiegazione: La traduzione è corretta perché mantiene il significato originale della frase, che descrive Aiace come un cavaliere franco e valoroso. L'unica modifica riguarda l'ordine delle parole, ma questo non cambia il senso della frase.

Alternativa: "E dall'altra parte Aiace era un cavaliere franco e valoroso."

Questa alternativa mantiene lo stesso significato della prima traduzione, ma utilizza una costruzione sintattica diversa per rendere la frase più fluida e naturale in italiano moderno.
Few-shot: Traduci le seguenti frasi dall'italiano antico all'italiano moderno.

Antico: "Ché 'l sol che v'era allor, or non è più."
Moderno: "Perché il sole che c'era allora, ora non c'è più."

Antico: "Poi che 'l mio duol non vuol mostrar altrui."
Moderno: "Dato che il mio 

In [ ]:
def judge(archaic, zero, few):
    prompt = f"""
Frase in italiano antico: {archaic}

Traduzione 1 (Zero-shot): {zero}
Traduzione 2 (Few-shot): {few}

Quale traduzione è più accurata e naturale in italiano moderno? Spiega brevemente la tua scelta.
"""
    return minerva(prompt, max_new_tokens=200, do_sample=False)[0]["generated_text"]


In [ ]:
archaic = "Et dall’altra parte Aiaces era uno cavaliere franco e prode all’arme."

zero = "E dall'altro lato Aiace era un cavaliere franco e valoroso."

few = "E dall'altra parte Aiace era un cavaliere franco e valoroso nelle armi."

# Run Minerva as a judge
judgment = judge(archaic, zero, few)

print(judgment)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Frase in italiano antico: Et dall’altra parte Aiaces era uno cavaliere franco e prode all’arme.

Traduzione 1 (Zero-shot): E dall'altro lato Aiace era un cavaliere franco e valoroso.
Traduzione 2 (Few-shot): E dall'altra parte Aiace era un cavaliere franco e valoroso nelle armi.

Quale traduzione è più accurata e naturale in italiano moderno? Spiega brevemente la tua scelta.

Risposta: La traduzione 2 è quella che meglio rispecchia il significato originale del testo, mantenendo l'ordine delle parole e utilizzando una costruzione sintattica coerente con l'italiano moderno. Inoltre, evita di introdurre elementi non presenti nel testo originale, come "nelle armi" o "prode".


Despite its strong performance, Minerva-7B occasionally made evaluative mistakes, especially when prompted to judge or refine its own translations. For example, it sometimes criticized the inclusion of terms that were, in fact, present in the original sentence (e.g., “nelle armi” or “prode”), revealing limitations in its reasoning capabilities. This behavior is consistent with the broader tendency of LLMs to simulate human judgment while lacking a grounded reference to the original input. Such errors highlight the model’s human-like, yet imperfect, interpretative behavior — a typical trait observed in LLM-based evaluations.

## STEP 2: Test with a prompt that have different samples

Conversational Turn

In [ ]:
def build_prompt_conversation(sentence):
    return [
        {
            "role": "system",
            "content": "Sei un linguista professionista specializzato in filologia e nella traduzione dall’italiano arcaico (**fiorentino – XIV–XV secolo**) all’italiano moderno. Il tuo compito è tradurre frasi scritte in **italiano arcaico** in **italiano moderno**, assicurandoti che il risultato sia: **Fedele al significato originale**, **Naturale e scorrevole**, **Chiaro e immediatamente comprensibile**, eliminando qualsiasi ambiguità o elemento stilistico obsoleto. Mantieni la struttura sintattica vicina all’originale **solo se** compatibile con quella dell’italiano moderno."
        },
        {
            "role": "user",
            "content": "Input: E però che la seconda suole talora per la grande provedenza fare timoroso, e la prima per l’ardire rendere altrui matto, assai utile è la presente distinzione."
        },
        {
            "role": "assistant",
            "content": "Output: Poiché la seconda, per la sua grande previdenza, può talora incutere timore, e la prima, per l’ardire, condurre alla follia, questa distinzione si rivela assai utile."
        },
        {
            "role": "user",
            "content": "Input: Egli, vedendo ciò, tacque, e con volto pietoso si trasse indietro."
        },
        {
            "role": "assistant",
            "content": "Output: Egli, vedendo ciò, tacque e con volto pietoso si ritrasse."
        },
        {
            "role": "user",
            "content": f"Input: {sentence}"
        }
    ]


Simple few shot

In [ ]:
def build_prompt(sentence, author=None, date=None):

    return f"""
Sei un esperto linguista umano, specializzato in filologia e traduzione dall'italiano antico all'italiano contemporaneo.
Dall'italiano antico traduci **solo** la frase in italiano moderno, in modo fluido e comprensibile, ma **senza spiegazioni o commenti**.

Esempio 1:
Antico: "Così che il core ne tremava tutto."
Moderno: "Il mio cuore tremava completamente."

Esempio 2:
Antico: "Li occhi miei, ché fermi stettero alquanto, pur pur mi si chiuser per paura."
Moderno: "I miei occhi, che rimasero fissi per un istante, alla fine si chiusero per la paura."

 Frase da tradurre:
"{sentence}"

 Traduzione:
"""


In [ ]:
prompt = build_prompt_conversation("Et dall’altra parte Aiaces era uno cavaliere franco e prode all’arme.")

output = minerva(prompt, max_new_tokens=60, do_sample=False)[0]["generated_text"]

print(output)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[{'role': 'system', 'content': 'Sei un linguista professionista specializzato in filologia e nella traduzione dall’italiano arcaico (**fiorentino – XIV–XV secolo**) all’italiano moderno. Il tuo compito è tradurre frasi scritte in **italiano arcaico** in **italiano moderno**, assicurandoti che il risultato sia: **Fedele al significato originale**, **Naturale e scorrevole**, **Chiaro e immediatamente comprensibile**, eliminando qualsiasi ambiguità o elemento stilistico obsoleto. Mantieni la struttura sintattica vicina all’originale **solo se** compatibile con quella dell’italiano moderno.'}, {'role': 'user', 'content': 'Input: E però che la seconda suole talora per la grande provedenza fare timoroso, e la prima per l’ardire rendere altrui matto, assai utile è la presente distinzione.'}, {'role': 'assistant', 'content': 'Output: Poiché la seconda, per la sua grande previdenza, può talora incutere timore, e la prima, per l’ardire, condurre alla follia, questa distinzione si rivela assai ut

##TEST WITH DATASET

In [ ]:
def build_prompt(sentence, author=None, date=None):

    return f"""
Sei un esperto linguista umano, specializzato in filologia e traduzione dall'italiano antico(Fiorentino 1300-1400) all'italiano moderno.
Dall'italiano antico traduci **solo** la frase in italiano moderno, mantenendo il significato originale e adattando la struttura all'italiano moderno.
**senza spiegazioni o commenti**

------
User: "Così che il core ne tremava tutto."
Minerva: "Il mio cuore tremava completamente."

------
User: "Li occhi miei, ché fermi stettero alquanto, pur pur mi si chiuser per paura."
Minerva: "I miei occhi, che rimasero fissi per un istante, alla fine si chiusero per la paura."

------
 User:
"{sentence}"

 Minerva:
"""


In [ ]:
import pandas as pd
import tqdm
from tqdm.auto import tqdm
tqdm.pandas()



df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_cleaned.csv")

df["zero_shot_prompt"] = df["Sentence"].apply(
    lambda s: f'Traduci solo la seguente frase dall\'italiano antico all\'italiano moderno, senza spiegazioni, senza commenti:\n\n{s}\n\n'
)


df["few_shot_prompt"] = df.apply(
    lambda row: build_prompt(row["Sentence"]),
    axis=1
)

def get_translation(prompt):
    response = minerva(prompt, max_new_tokens=60, do_sample=False)[0]["generated_text"]
    return response.replace(prompt, "").strip().split("\n")[0]

df["zero_shot_translation"] = df["zero_shot_prompt"].progress_apply(get_translation)
df["few_shot_translation"] = df["few_shot_prompt"].progress_apply(get_translation)

df.to_csv("dataset_with_translations15to35.csv", index=False)

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 After analyzing the output, I realized that a more carefully designed prompt was necessary to achieve better results.

#Minerva-7b with a simple prompt and two samples

In [ ]:
import pandas as pd
import json
import torch
from tqdm.auto import tqdm
from transformers import pipeline

tqdm.pandas()

In [ ]:
minerva = pipeline(
    task="text-generation",
    model="sapienzanlp/Minerva-7B-instruct-v1.0",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_cleaned.csv")

def build_prompt_conversation(sentence):
    return f"""
Sei un esperto linguista umano, specializzato in filologia e traduzione dall'italiano antico(Fiorentino 1300-1400) all'italiano moderno.
Dall'italiano antico traduci **solo** la frase in italiano moderno, mantenendo il significato originale e adattando la struttura all'italiano moderno.
**senza spiegazioni o commenti**

------
User: "Così che il core ne tremava tutto."
Minerva: "Il mio cuore tremava completamente."

------
User: "Li occhi miei, ché fermi stettero alquanto, pur pur mi si chiuser per paura."
Minerva: "I miei occhi, che rimasero fissi per un istante, alla fine si chiusero per la paura."

------
 User:
"{sentence}"

 Minerva:
"""


def get_translation(prompt):
    output = minerva(prompt, max_new_tokens=100, do_sample=False)[0]["generated_text"]
    response = output.replace(prompt, "").strip()
    if response.lower().startswith("output:"):
        response = response[7:].strip()
    return response


df["prompt"] = df["Sentence"].apply(build_prompt_conversation)
df["modern_translation"] = df["prompt"].progress_apply(get_translation)


output_path = "/content/minerva_few_shot_simple_prompt.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        json.dump({
            "author": row["Author"],
            "date": row["Date"],
            "region": row["Region"],
            "archaic_sentence": row["Sentence"],
            "modern_translation": row["modern_translation"]
        }, f, ensure_ascii=False)
        f.write("\n")

print(f" File JSONL saved: {output_path}")

cleaning

In [ ]:
import json
import re

# Percorso al file da pulire
input_path = "/content/minerva_few_shot_simple_prompt.jsonl"
output_path = "/content/minerva_few_shot_simple_prompt_cleaned.jsonl"

def clean_translation(text):
    # Tieni solo la parte prima di "Utente:" o "Assistente:" o varianti
    return re.split(r"\n?Utente:|\n?Assistente:", text)[0].strip()

# Legge, pulisce e riscrive il file
with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        item = json.loads(line)
        item["modern_translation"] = clean_translation(item["modern_translation"])
        json.dump(item, outfile, ensure_ascii=False)
        outfile.write("\n")

print(f" File ripulito salvato in: {output_path}")

converting into csv for analysis

In [ ]:
import pandas as pd

# Percorso al file .jsonl da convertire
input_jsonl = "/content/minerva_few_shot_simple_prompt_cleaned.jsonl"
output_csv = "/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_minerva_few_shot_simple_prompt.csv"

# Carica il file .jsonl
df = pd.read_json(input_jsonl, lines=True)

# Salva in formato .csv
df.to_csv(output_csv, index=False)

print(f" File CSV salvato in: {output_csv}")

In [ ]:
import pandas as pd
import json
df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_minerva_few_shot_simple_prompt.csv")

#convert in jsonl
jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["archaic_sentence"],
        "Translated Sentence": row["modern_translation"]
    })

out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-Minerva-7b-instruct(simple_prompt_fewshot).jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

#Minerva-7b with a prompt more detailed and two samples

In [ ]:
import pandas as pd
import json
import torch
from tqdm.auto import tqdm
from transformers import pipeline

tqdm.pandas()

In [ ]:
minerva = pipeline(
    task="text-generation",
    model="sapienzanlp/Minerva-7B-instruct-v1.0",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_cleaned.csv")



def build_prompt_conversation(sentence):
    return f"""Sistema: Sei un linguista professionista specializzato in filologia e nella traduzione dall’italiano arcaico (**fiorentino – XIV–XV secolo**) all’italiano moderno. Il tuo compito è tradurre frasi scritte in **italiano arcaico** in **italiano moderno**, assicurandoti che il risultato sia: **Fedele al significato originale**, **Naturale e scorrevole**, **Chiaro e comprensibile**, eliminando qualsiasi ambiguità o elemento stilistico obsoleto. Se necessario cambia la struttura sintattica, mantenendo il significato e il tono invariati.

Utente: Input: E però che la seconda suole talora per la grande provedenza fare timoroso, e la prima per l’ardire rendere altrui matto, assai utile è la presente distinzione.
Assistente: Poiché la seconda, per la sua grande previdenza, può talora incutere timore, e la prima, per l’ardire, condurre alla follia, questa distinzione si rivela assai utile.

Utente: Input: Egli, vedendo ciò, tacque, e con volto pietoso si trasse indietro.
Assistente: Egli, vedendo ciò, tacque e con volto pietoso si ritrasse.

Utente: Input: {sentence}
Assistente:"""


def get_translation(prompt):
    output = minerva(prompt, max_new_tokens=100, do_sample=False)[0]["generated_text"]
    response = output.replace(prompt, "").strip()
    if response.lower().startswith("output:"):
        response = response[7:].strip()
    return response


df["prompt"] = df["Sentence"].apply(build_prompt_conversation)
df["modern_translation"] = df["prompt"].progress_apply(get_translation)


output_path = "/content/minerva__few_shot_detailed_prompt.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        json.dump({
            "author": row["Author"],
            "date": row["Date"],
            "region": row["Region"],
            "archaic_sentence": row["Sentence"],
            "modern_translation": row["modern_translation"]
        }, f, ensure_ascii=False)
        f.write("\n")

print(f" File JSONL salvato: {output_path}")


  0%|          | 0/17 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignore

 File JSONL salvato: /content/minerva_batch_80to97.jsonl


cleaning

In [ ]:
import json
import re

# Percorso al file da pulire
input_path = "/content/minerva__few_shot_detailed_prompt.jsonl"
output_path = "/content/minerva__few_shot_detailed_prompt_cleaned.jsonl"

def clean_translation(text):
    # Tieni solo la parte prima di "Utente:" o "Assistente:" o varianti
    return re.split(r"\n?Utente:|\n?Assistente:", text)[0].strip()

# Legge, pulisce e riscrive il file
with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        item = json.loads(line)
        item["modern_translation"] = clean_translation(item["modern_translation"])
        json.dump(item, outfile, ensure_ascii=False)
        outfile.write("\n")

print(f" File ripulito salvato in: {output_path}")


 File ripulito salvato in: /content/minerva_four_cleaned.jsonl


converting into csv for the analysis

In [ ]:
import pandas as pd

# Percorso al file .jsonl da convertire
input_jsonl = "/content/minerva__few_shot_detailed_prompt_cleaned.jsonl"
output_csv = "/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_minerva_few_shot_detailed_prompt.csv"

# Carica il file .jsonl
df = pd.read_json(input_jsonl, lines=True)

# Salva in formato .csv
df.to_csv(output_csv, index=False)

print(f" File CSV salvato in: {output_csv}")


 File CSV salvato in: /content/minerva_55to97_cleaned.csv


<ipython-input-6-4126690898>:8: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df = pd.read_json(input_jsonl, lines=True)
<ipython-input-6-4126690898>:8: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df = pd.read_json(input_jsonl, lines=True)
<ipython-input-6-4126690898>:8: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the 

In [ ]:
import pandas as pd
import json
df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_minerva_few_shot_detailed_prompt.csv")

#convert in jsonl
jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["archaic_sentence"],
        "Translated Sentence": row["modern_translation"]
    })

out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-Minerva-7b-instruct(detailed_prompt_fewshot).jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

#Minerva-7b - detailed prompt with four optimized samples
Example 1 - Structure: Structure: consecutive clause + postponed subject;
Example 2 - Structure: causal clause + archaic adjectives;
Example 3 - Structure: Main impersonal clause and comparative adverbial phrase;
Example 4 - Structure: Impersonal past tense + dependent (subordinate) clause.

In [ ]:
import pandas as pd
import json
import torch
from tqdm.auto import tqdm
from transformers import pipeline

tqdm.pandas()

In [ ]:
# Inizializza la pipeline
minerva = pipeline(
    task="text-generation",
    model="sapienzanlp/Minerva-7B-instruct-v1.0",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/dataset_cleaned.csv")

def build_prompt_conversation(sentence):
    return f"""Sistema: Sei un linguista professionista specializzato in filologia e nella traduzione dall’italiano arcaico (**fiorentino – XIV–XV secolo**) all’italiano moderno. Il tuo compito è tradurre frasi scritte in **italiano arcaico** in **italiano moderno**, assicurandoti che il risultato sia: **Fedele al significato originale**, **Naturale e scorrevole**, **Chiaro e comprensibile**, eliminando qualsiasi ambiguità o elemento stilistico obsoleto. Se necessario cambia la struttura sintattica, mantenendo il significato e il tono invariati.

Utente: Input: E però che la moltitudine non sapea discernere il vero dal falso, si lasciò menare per inganno.
Assistente: E poiché la moltitudine non sapeva distinguere il vero dal falso, si lasciò facilmente ingannare.

Utente: Input: Niuno savio uomo così leggermente si lascia trasportare dalla ventura.
Assistente: Nessun uomo saggio si lascia trasportare con leggerezza dalla sorte.

Utente: Input: Parve a molti che colui avesse parlato con arte più che con verità.
Assistente: Parve a molti che egli avesse parlato più con astuzia che con sincerità.

Utente: Input: Coloro che il senno lor non adoperano a buono fine, degni sono di misera sorte.
Assistente: Coloro che non usano la loro intelligenza per un fine giusto sono degni di una sorte miserabile.

Utente: Input: {sentence}
Assistente:"""



def get_translation(prompt):
    output = minerva(prompt, max_new_tokens=100, do_sample=False)[0]["generated_text"]
    response = output.replace(prompt, "").strip()
    if response.lower().startswith("output:"):
        response = response[7:].strip()
    return response


df["prompt"] = df["Sentence"].apply(build_prompt_conversation)
df["modern_translation"] = df["prompt"].progress_apply(get_translation)


output_path = "/content/minerva_few_shot_detailed_4_optimized.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        json.dump({
            "author": row["Author"],
            "date": row["Date"],
            "region": row["Region"],
            "archaic_sentence": row["Sentence"],
            "modern_translation": row["modern_translation"]
        }, f, ensure_ascii=False)
        f.write("\n")

print(f" File JSONL salvato: {output_path}")


  0%|          | 0/42 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignore

 File JSONL salvato: /content/minerva_55to97.jsonl


cleaning

In [ ]:
import json
import re

# Percorso al file da pulire
input_path = "/content/minerva_few_shot_detailed_4_optimized.jsonl"
output_path = "/content/minerva_few_shot_detailed_4_optimized_cleaned.jsonl"

def clean_translation(text):
    # Tieni solo la parte prima di "Utente:" o "Assistente:" o varianti
    return re.split(r"\n?Utente:|\n?Assistente:", text)[0].strip()

# Legge, pulisce e riscrive il file
with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        item = json.loads(line)
        item["modern_translation"] = clean_translation(item["modern_translation"])
        json.dump(item, outfile, ensure_ascii=False)
        outfile.write("\n")

print(f" File ripulito salvato in: {output_path}")

converting into csv for the analysis

In [ ]:
import pandas as pd

# Percorso al file .jsonl da convertire
input_jsonl = "/content/minerva_few_shot_detailed_4_optimized_cleaned.jsonl"
output_csv = "/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_minerva_few_shot_detailed_4_optimized.csv"

# Carica il file .jsonl
df = pd.read_json(input_jsonl, lines=True)

# Salva in formato .csv
df.to_csv(output_csv, index=False)

print(f" File CSV salvato in: {output_csv}")

In [ ]:
import pandas as pd
import json
df = pd.read_csv("/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/DATASET/dataset_with_minerva_few_shot_detailed_4_optimized.csv")

#convert in jsonl
jsonl_data = []
for _, row in df.iterrows():
    jsonl_data.append({
        "Sentence": row["archaic_sentence"],
        "Translated Sentence": row["modern_translation"]
    })

out_dir="/content/drive/MyDrive/AleBERTs&FraBERT_shared_folder_HW2/jsonl_files/AleBERTs&FraBERT-hw2_transl-Minerva-7b-instruct(4_optimized_fewshot).jsonl"
with open(out_dir, "w", encoding="utf-8") as f:
    for item in jsonl_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")